<a href="https://colab.research.google.com/github/uniquing/API-Connection/blob/main/API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 串接政府AQI空氣資料


In [ ]:
# Update the module: requests and lxml
pip install -U requests lxml

In [2]:
import json
# Here try to use different module to request the url
import urllib.request as request


# src means where the source come from
src="https://data.epa.gov.tw/api/v1/aqx_p_432?limit=1000&api_key=9be7b239-557b-4c10-9775-78cadfc555e9&format=json"

# Use the google developer tools to see the data is json, then use the module json to load
with request.urlopen(src) as response:
  data=json.load(response)

In [5]:
import pandas as pd

# According to the date we retrieved what we want
# Location, aqi and PM2.5
dlist=data['records']
counties=[d["County"] for d in dlist]
sites=[d["SiteName"] for d in dlist]
aqi= [d["AQI"] for d in dlist]
pm25=[d["PM2.5"] for d in dlist]

# Load the data into dictionary, then to dataframe for late use
aq_dict={
    "County": counties,
    "SiteName": sites,
    "AQI": aqi,
    "PM2.5": pm25 
}
df=pd.DataFrame(aq_dict)

In [ ]:
import chardet
import requests

# Use chardet to detect what the data enconded like 'utf-8'
# In case if mandarin become garbeld
r=requests.get(src)
print(chardet.detect(r.content))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [6]:
# Tranform the df into csv or json fot other use
df.to_csv("AQI.csv",encoding="utf-8",index=False)
with open("AQI.json","w") as f:
  json.dump(aq_dict, f, ensure_ascii=False)

### Dcard熱門前10


In [7]:
import requests
import re
# Use BeautifulSoup on html type website
# Just like lxml on XML type and module json on json type
from bs4 import BeautifulSoup

dcard_topic_page = 'https://www.dcard.tw/f'
resp = requests.get(dcard_topic_page)
soup = BeautifulSoup(resp.text, 'html.parser')

In [26]:
# Use the CSSselector to help me locate the content I want
Dcard_title_css = '.ebwnQU'
title_tag = soup.select(Dcard_title_css)
title_list = [str(i)+"."+e.text for i, e in enumerate(title_tag) if (i<=9)]
title_list

['0.#更 強者我朋友 一年內從87瘦到56',
 '1.我媽在餐廳撩了服務生，結果..',
 '2.不想賺髒錢，我辭職了！',
 '3.更/是我瘋了 還是整間公司都瘋了',
 '4.說女友跟狗一樣被分手….',
 '5.#更新一張被已讀/莫名其妙自以為全世界都圍著她轉的網紅？',
 '6.Lisa成為MAC全球代言人💗',
 '7.冰旋風被加爆真的好爽',
 '8.一定要看！  超準！ 12星座連自己都討厭的缺點',
 '9.初次見面不能上床？']

### 有API的爬蟲

In [27]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from datetime import datetime

In [28]:
# User-Agent to simulate real human surf Internet
# Cookie infromation be used for avoiding ip lock
# Or you can use sleep method in request module.

headers = {'cookie': 'ECC=GoogleBot',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}


In [29]:
# query try to put the thing you want to search 
# and pages for how mant pages you want to load
query='macbook'
pages=10

In [30]:
#Pchome has two api, this v3.3 help you find the product_id
Id_list=[]
for page in list(range(1,pages+1)):
  url='https://ecshweb.pchome.com.tw/search/v3.3/all/results?q={}&page={}&sort=sale/dc'.format(query,page)
  resp=requests.get(url, headers=headers)
  for e in resp.json()['prods']:
    Id_list.append(e['Id'])
  print("There are {} products in list, when p.{}".format(len(Id_list),page))


There are 20 products in list, when p.1
There are 40 products in list, when p.2
There are 60 products in list, when p.3
There are 80 products in list, when p.4
There are 100 products in list, when p.5
There are 120 products in list, when p.6
There are 140 products in list, when p.7
There are 160 products in list, when p.8
There are 180 products in list, when p.9
There are 200 products in list, when p.10


In [31]:
# You can use product_id on another api to exact what the produt like 
Id_list[0:5]

['DYAJEX-1900AOLWE',
 'DYAJCS-1900AQM6V',
 'DYAJET-1900ARL8U',
 'DYAJEX-1900APCGM',
 'DYAJCS-1900AQM5G']

### Try weatherstack api

In [33]:
# This is the open api for world weather
# You can register for free and get your own API_key

import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from time import sleep
from datetime import datetime

# The query would be the city I want to know
API_key='your own key' #Insert your's here
querys=['Taipei','Tokyo','Seoul','Bejing','Singapore']


In [34]:
df=pd.DataFrame()
for q in querys:
  url="http://api.weatherstack.com/current?access_key={}&query={}".format(API_key,q)
  resp=requests.get(url)
  resp_=resp.json()
  dict_={
        'country':resp_['location']['country'],
        'lat':resp_['location']['lat'],
        'lon':resp_['location']['lon'],
        'wdescription':resp_['current']['weather_descriptions'],
        'wcode':resp_['current']['weather_code'],
        'temprature':resp_['current']['temperature'],
        'pressure':resp_['current']['pressure'],
        'humidity':resp_['current']['humidity'],
        'feelslike':resp_['current']['feelslike'],
        'cloudcover':resp_['current']['cloudcover'],
        'observation_time':resp_['current']['observation_time'],
        'localtime':resp_['location']['localtime']
        }
  df_ = pd.DataFrame(dict_)
  df = pd.concat([df,df_], ignore_index=True)
  print("Scraping {} data".format(q))


Scraping Taipei data
Scraping Tokyo data
Scraping Seoul data
Scraping Bejing data
Scraping Singapore data


In [35]:
df

,country,lat,lon,wdescription,wcode,temprature,pressure,humidity,feelslike,cloudcover,observation_time,localtime
0,Taiwan,25.039,121.525,Light Drizzle,296,25,1013,89,27,75,08:44 AM,2020-10-20 16:44
1,Japan,35.690,139.692,Partly cloudy,116,21,1023,38,21,10,08:44 AM,2020-10-20 17:44
2,South Korea,37.566,127.000,Sunny,113,25,1025,30,25,0,08:44 AM,2020-10-20 17:44
3,Indonesia,-7.491,110.986,Cloudy,119,32,1009,49,36,81,08:44 AM,2020-10-20 15:44
4,Singapore,1.293,103.856,Partly cloudy,116,29,1006,79,32,75,08:44 AM,2020-10-20 16:44
